## Robustness of NeuralHash to Some Standard Image Processing Transformations

In [ ]:
from data import IMAGENETTE
import utils

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os

In [ ]:
folder_path = '../images/'
images = IMAGENETTE()
x = images.load()
images.save_to_disk(x, folder_path, num_images=100)

### Apply the image transformations to the Images

In [ ]:
def flip_image(folder_path, file_name):
    if 'flipped' in file_name or 'rotated' in file_name:
        return
    # Open the image file
    image_path = os.path.join(folder_path, file_name)
    image = Image.open(image_path)

    # Flip the image horizontally
    flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # Save the flipped image with a new file name
    flipped_image_path = os.path.join(folder_path, f"{file_name.split('.')[0]}_flipped.{file_name.split('.')[1]}")
    flipped_image.save(flipped_image_path)

    # Close the original image
    image.close()
    return os.path.join(folder_path, f"{file_name.split('.')[0]}_flipped.{file_name.split('.')[1]}")

In [ ]:
def rotate_image(folder_path, file_name):
    if 'flipped' in file_name or 'rotated' in file_name:
        return
    # Open the image file
    image_path = os.path.join(folder_path, file_name)
    image = Image.open(image_path)

    # Rotate the image by 90 degrees clockwise
    rotated_image = image.transpose(Image.ROTATE_90)

    # Save the rotated image with a new file name
    rotated_image_path = os.path.join(folder_path, f"{file_name.split('.')[0]}_rotated.{file_name.split('.')[1]}")
    rotated_image.save(rotated_image_path)

    # Close the original image
    image.close()
    return os.path.join(folder_path, f"{file_name.split('.')[0]}_rotated.{file_name.split('.')[1]}")

In [ ]:
original_paths, flipped_paths, rotated_paths = [], [], []
original_hashes, flipped_hashes, rotated_hashes = [], [], []
for file in sorted(os.listdir(folder_path)):
    if file.endswith(('.jpg', '.jpeg', '.png')):
        original_paths.append(os.path.join(folder_path, file))
        if original_paths[-1] is not None:
            original_hashes.append(utils.compute_hash(original_paths[-1], hash_file_path='./nhcalc'))
        
        flipped_paths.append(flip_image(folder_path, file))    
        if flipped_paths[-1] is None:
            flipped_paths.pop(-1)
        else:
            flipped_hashes.append(utils.compute_hash(flipped_paths[-1], hash_file_path='./nhcalc'))
        

        rotated_paths.append(rotate_image(folder_path, file))    
        if rotated_paths[-1] is None:
            rotated_paths.pop(-1)
        else:
            rotated_hashes.append(utils.compute_hash(rotated_paths[-1], hash_file_path='./nhcalc'))

In [ ]:
l2_threshold = 20
hamming_threshold = 40

### Compute the Hamming Distances and L2 Distortion between the Hashes of the Flipped Sets of Images

In [ ]:
# Original and flipped
success = []
for orig_path, flipped_path, orig_hash, flipped_hash in zip(original_paths, flipped_paths, original_hashes, flipped_hashes):
    orig_img, flipped_img = utils.load_img(orig_path), utils.load_img(flipped_path)
    hamming_dist, l2_dist = utils.distance(orig_hash, flipped_hash, "hamming"), utils.distance(orig_img, flipped_img, "l2")
    print(f'{orig_path} & {flipped_path}\t Hamming Distance: {hamming_dist/96 * 100:.2f}%\t L2 Dist: {l2_dist:.2f}')
    if hamming_dist >= hamming_threshold and l2_dist < l2_threshold:
        success.append(1)
    else:
        success.append(0)

In [ ]:
flipped_asr = sum(success)/len(success)
print(f'ASR: {100*flipped_asr:.2f}%')

### Compute the Hamming Distances and L2 Distortion between the Hashes of the Rotated Sets of Images

In [ ]:
# Original and rotated
for orig_path, rotated_path, orig_hash, rotated_hash in zip(original_paths, rotated_paths, original_hashes, rotated_hashes):
    orig_img, rotated_img = utils.load_img(orig_path), utils.load_img(rotated_path)
    hamming_dist, l2_dist = utils.distance(orig_hash, rotated_hash, "hamming"), utils.distance(orig_img, rotated_img, "l2")
    print(f'{orig_path} & {rotated_path}\t Hamming Distance: {hamming_dist/96 * 100:.2f}%\t L2 Dist: {l2_dist:.2f}')
    if hamming_dist >= hamming_threshold and l2_dist < l2_threshold:
        success.append(1)
    else:
        success.append(0)

In [ ]:
rotated_asr = sum(success)/len(success)
print(f'ASR: {100*rotated_asr:.2f}%')